# Cálculo de métricas

## Instalação de dependências e leitura de dados brutos

Os dados brutos de repositórios e _pull requests_ mineradas a partir do GitHub devem estar inseridos no diretório `/dataset/raw/`.

Devem obrigatoriamente conter os seguintes campos:
- **repositories.csv**: 'RepositoryId', 'OwnerName', 'RepositoryName', 'PrimaryLanguage', 'PullRequestsCount', 'StargazersCount', 'HasDependabotPRs', 'QtyDependabotPRs'
- **prs.csv**: 'CreatedAt', 'ClosedAt', 'Author', 'RepositoryId', 'State', 'PrimaryLanguage'

In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd

prs_file_path = '../dataset/raw/prs.csv'
prs_raw_data = pd.read_csv(prs_file_path, header=0)

prs = prs_raw_data[['CreatedAt', 'ClosedAt', 'Author', 'RepositoryId', 'State', 'PrimaryLanguage']]

repos_file_path = '../dataset/raw/repositories.csv'
repos_raw_data = pd.read_csv(repos_file_path, header=0)

repos = repos_raw_data[['RepositoryId', 'RepositoryName', 'OwnerName', 'HasDependabotPRs', 'QtyDependabotPRs']]

## Métricas de PRs do Dependabot por projeto (aceitos, rejeitados, abertos)

Gera um novo arquivo chamado `dependabot_pr_statuses_by_repo.csv`, contendo as seguintes colunas para cada repositório:

- **RepositoryId**: identificador único do repositório.
- **RepositoryName**: título do repositório.
- **OwnerName**: nome do proprietário do repositório.
- **accepted**: quantidade de PRs do Dependabot aceitos (`MERGED`) naquele projeto.
- **rejected**: quantidade de PRs do Dependabot rejeitados (`CLOSED`) naquele projeto.
- **opened**: quantidade de PRs do Dependabot ainda abertos (`OPEN`) naquele projeto.
- **AcceptedByProtejectPercentage**: porcentagem de PRs do Dependabot que aceitos (`MERGED`) naquele projeto.

In [ ]:
dependabot_prs = prs[prs['Author'].isin(['dependabot'])]

file_path = '../dataset/processed/output/dependabot_pr_statuses_by_repo.csv'

accepted = dependabot_prs[dependabot_prs['State'] == 'MERGED'].groupby('RepositoryId').size().rename('Accepted')
rejected = dependabot_prs[dependabot_prs['State'] == 'CLOSED'].groupby('RepositoryId').size().rename('Rejected')
opened = dependabot_prs[dependabot_prs['State'] == 'OPEN'].groupby('RepositoryId').size().rename('Open')

summary = pd.concat([accepted, rejected, opened], axis=1).fillna(0).astype(int).reset_index()
summary = summary.merge(repos[['RepositoryId', 'RepositoryName', 'OwnerName']], on='RepositoryId', how='left')

summary['AcceptedByProjectPercentage'] = (
    summary['Accepted'] / (summary['Accepted'] + summary['Rejected'] + summary['Open'])
) * 100
summary['AcceptedByProjectPercentage'] = summary['AcceptedByProjectPercentage'].round(2)

summary[['RepositoryId', 'RepositoryName', 'OwnerName', 'Accepted', 'Rejected', 'Open', 'AcceptedByProjectPercentage']].to_csv(file_path, index=False, sep=';', decimal=',')

print(f"Arquivo atualizado em: {file_path}")


Arquivo atualizado em:../dataset/processed/output/dependabot_pr_statuses_by_repo.csv


## Métricas de PRs do Dependabot por linguagem de programação

Gera um novo arquivo chamado `dependabot_prs_created_by_language.csv`, contendo as seguintes colunas para cada linguagem de programação:

- **language**: nome da linguagem de programação atribuída aos PRs.
- **pullRequestsNumbers**: número total de PRs do Dependabot criados para aquela linguagem.
- **pullRequestsNumberPercentage**: porcentagem que representa o total de PRs daquela linguagem em relação ao total geral de PRs do Dependabot, considerando apenas linguagens conhecidas (excluindo "Desconhecida").

As linguagens populares são priorizadas e ordenadas na seguinte sequência: `['Python', 'JavaScript', 'TypeScript', 'Go', 'Java', 'Rust', 'C++', 'HTML', 'Shell', 'Jupyter Notebook', 'C']`.

PRs categorizados como "Desconhecida" (que não possuem linguagem de programação envolvida) são excluídos dos cálculos e da soma de linguagens não populares para garantir maior precisão.

In [ ]:
dependabot_prs.loc[:, 'PrimaryLanguage'] = dependabot_prs['PrimaryLanguage'].fillna("Desconhecida")

total_prs = len(dependabot_prs)

grouped_by_language = dependabot_prs.groupby('PrimaryLanguage').size().rename('pullRequestsNumbers').reset_index()
grouped_by_language = grouped_by_language.rename(columns={'PrimaryLanguage': 'language'})

grouped_by_language['pullRequestsNumberPercentage'] = round(
    (grouped_by_language['pullRequestsNumbers'] / total_prs) * 100, 2
)

top3 = grouped_by_language.sort_values(by='pullRequestsNumbers', ascending=False).head(3).reset_index(drop=True)

min_lang = grouped_by_language.loc[grouped_by_language['pullRequestsNumbers'].idxmin()]

prioritized_languages = ['Python', 'JavaScript', 'TypeScript', 'Go', 'Java', 'Rust', 'C++', 'HTML', 'Shell', 'Jupyter Notebook', 'C']

non_popular_mask = (~grouped_by_language['language'].isin(prioritized_languages)) & (grouped_by_language['language'] != 'Desconhecida')
non_popular_total = grouped_by_language[non_popular_mask]['pullRequestsNumbers'].sum()

total_prs_known = grouped_by_language[grouped_by_language['language'] != 'Desconhecida']['pullRequestsNumbers'].sum()
non_popular_percentage = round((non_popular_total / total_prs_known) * 100, 2)

grouped_by_language['priority_order'] = grouped_by_language['language'].apply(
    lambda lang: prioritized_languages.index(lang) if lang in prioritized_languages else len(prioritized_languages)
)
grouped_by_language = grouped_by_language.sort_values(by='priority_order').drop(columns='priority_order').reset_index(drop=True)

output_path = '../dataset/processed/output/dependabot_prs_created_by_language.csv'
grouped_by_language.to_csv(output_path, index=False, sep=';', decimal=',')

## Métricas de tempo de aceitação de PRs do Dependabot

Gera um novo arquivo chamado `prs_by_time.csv`, contendo dados de tempo de aceitação dos pull requests (PRs) criados pelo Dependabot que foram aceitos (`MERGED`). O arquivo inclui as seguintes colunas para cada PR:

- **RepositoryId**: identificador único do repositório.
- **RepositoryName**: nome do repositório onde o PR foi aberto.
- **CreatedAt**: data e hora em que o PR foi criado.
- **MergedAt**: data e hora em que o PR foi aceito (merge).
- **PublishedAt**: data e hora em que o PR foi publicado.
- **ClosedAt**: data e hora em que o PR foi fechado.
- **merge_duration_seconds**: tempo total, em segundos, entre a criação e a aceitação do PR.
- **merge_duration_minutes**: tempo total, em minutos inteiros, entre a criação e a aceitação do PR.
- **merge_duration_hours**: tempo total, em horas inteiras, entre a criação e a aceitação do PR.

Somente PRs com datas válidas e em que a data de aceitação é posterior à data de criação são considerados. O objetivo é analisar quanto tempo os PRs do Dependabot levam, em média, para serem aceitos, bem como identificar casos extremos de tempo mínimo e máximo de aceitação.

In [14]:
prs_data = pd.read_csv(prs_file_path, parse_dates=['CreatedAt', 'MergedAt', 'ClosedAt'])

dependabot_prs = prs_data[prs_data['Author'].isin(['dependabot'])].copy()

merged_prs = dependabot_prs[
    (dependabot_prs['State'] == 'MERGED') &
    dependabot_prs['MergedAt'].notnull() &
    dependabot_prs['CreatedAt'].notnull() &
    (dependabot_prs['MergedAt'] > dependabot_prs['CreatedAt'])
].copy()


merged_prs['merge_duration_seconds'] = (merged_prs['MergedAt'] - merged_prs['CreatedAt']).dt.total_seconds().astype(int)
merged_prs['merge_duration_minutes'] = (merged_prs['merge_duration_seconds'] // 60).astype(int)
merged_prs['merge_duration_hours'] = (merged_prs['merge_duration_seconds'] // 3600).astype(int)
merged_prs['merge_duration_days'] = (merged_prs['MergedAt'] - merged_prs['CreatedAt']).dt.days

columns_to_export = [
    'RepositoryId', 'RepositoryName', 'CreatedAt', 'MergedAt', 'PublishedAt', 'ClosedAt',
    'merge_duration_seconds', 'merge_duration_minutes', 'merge_duration_hours', 'merge_duration_days'
]
merged_prs_export = merged_prs[columns_to_export]


export_path = '../dataset/processed/output/prs_by_time.csv'
merged_prs_export.to_csv(export_path, index=False, sep=';', decimal=',')

print(f"📁 Arquivo 'prs_by_time.csv' gerado com sucesso em: {export_path}")
print(f"✅ Total de PRs exportados: {len(merged_prs_export)}")

📁 Arquivo 'prs_by_time.csv' gerado com sucesso em: ../dataset/processed/output/prs_by_time.csv
✅ Total de PRs exportados: 23980


## Métricas de tempo para PRs do Dependabot

Processa os dados de PRs do Dependabot para calcular métricas relacionadas ao tempo de aceitação e fechamento dos PRs, gerando informações sobre:
- PR com maior tempo para ser aceito (tempo entre criação e merge), apresentado em meses, junto com o nome do projeto correspondente.
- PR com menor tempo para ser aceito, detalhado em horas, minutos e segundos, junto com o nome do projeto.
- Tempo médio em dias para fechar um PR (tempo entre criação e fechamento), considerando todos os PRs do Dependabot que foram fechados.

O cálculo considera apenas PRs filtrados para o autor "dependabot" e valida que as datas de criação, merge e fechamento são válidas e coerentes (merge e fechamento posteriores à criação).

In [ ]:
prs_data = pd.read_csv(prs_file_path, parse_dates=['CreatedAt', 'MergedAt', 'ClosedAt'])
prs_data = prs_data[prs_data['Author'].isin(['dependabot'])].copy()

merged_prs = prs_data[(prs_data['State'] == 'MERGED') & (prs_data['MergedAt'].notnull()) & (prs_data['CreatedAt'].notnull())].copy()

merged_prs = merged_prs[merged_prs['MergedAt'] > merged_prs['CreatedAt']]
merged_prs['merge_duration_seconds'] = (merged_prs['MergedAt'] - merged_prs['CreatedAt']).dt.total_seconds()

max_merge = merged_prs.loc[merged_prs['merge_duration_seconds'].idxmax()]
max_merge_months = int(max_merge['merge_duration_seconds'] / (60 * 60 * 24 * 30))

min_merge = merged_prs.loc[merged_prs['merge_duration_seconds'].idxmin()]
min_seconds = int(min_merge['merge_duration_seconds'])
min_hours = min_seconds // 3600
min_minutes = (min_seconds % 3600) // 60
min_secs = min_seconds % 60

closed_prs = prs_data[prs_data['ClosedAt'].notnull() & prs_data['CreatedAt'].notnull() & (pd.to_datetime(prs_data['ClosedAt']) > pd.to_datetime(prs_data['CreatedAt']))].copy()

closed_prs['close_duration_days'] = (closed_prs['ClosedAt'] - closed_prs['CreatedAt']).dt.total_seconds() / (60 * 60 * 24)
avg_close_time = int(closed_prs['close_duration_days'].mean())

print(f"PR com MAIOR tempo para ser aceito: {max_merge_months} meses")
print(f"    Projeto: {max_merge['RepositoryName']}")
print(f"\nPR com MENOR tempo para ser aceito: {min_hours} horas, {min_minutes} minutos e {min_secs} segundos")
print(f"    Projeto: {min_merge['RepositoryName']}")
print(f"\nTempo MÉDIO em dias para fechar um PR: {avg_close_time} dias")
